In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [2]:
data = pd.read_csv('/5cb9119d-9763-4604-bc0b-0d5bbf758c2e-Grammar Correction.csv')

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Ungrammatical Statement'] + data['Standard English'])

In [4]:
input_sequences = tokenizer.texts_to_sequences(data['Ungrammatical Statement'])
target_sequences = tokenizer.texts_to_sequences(data['Standard English'])

In [5]:
max_len = max(max(len(seq) for seq in input_sequences), max(len(seq) for seq in target_sequences))
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_len, padding='post')

In [6]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256
latent_dim = 512

In [7]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [8]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [9]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [10]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [11]:
target_sequences = np.expand_dims(target_sequences, -1)

In [13]:
model.fit([input_sequences, input_sequences], target_sequences, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
26/26 [==============================] - 36s 1s/step - loss: 2.6225 - val_loss: 3.2032
Epoch 2/50
26/26 [==============================] - 37s 1s/step - loss: 2.5467 - val_loss: 3.1689
Epoch 3/50
26/26 [==============================] - 38s 1s/step - loss: 2.5051 - val_loss: 3.0466
Epoch 4/50
26/26 [==============================] - 38s 1s/step - loss: 2.4682 - val_loss: 3.0811
Epoch 5/50
26/26 [==============================] - 35s 1s/step - loss: 2.4109 - val_loss: 3.0059
Epoch 6/50
26/26 [==============================] - 38s 1s/step - loss: 2.3873 - val_loss: 3.0222
Epoch 7/50
26/26 [==============================] - 35s 1s/step - loss: 2.3427 - val_loss: 2.9695
Epoch 8/50
26/26 [==============================] - 38s 1s/step - loss: 2.3075 - val_loss: 2.9853
Epoch 9/50
26/26 [==============================] - 35s 1s/step - loss: 2.2761 - val_loss: 3.0102
Epoch 10/50
26/26 [==============================] - 38s 1s/step - loss: 2.2563 - val_loss: 2.9625
Epoch 11/50
26/26 [

In [14]:
model.save('grammar_correction_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model = tf.keras.models.load_model('grammar_correction_model.h5')

In [16]:
encoder_inputs = model.input[0]  # input_1
encoder_emb = model.layers[2](encoder_inputs)  # embedding_1
_, state_h_enc, state_c_enc = model.layers[4](encoder_emb)  # lstm_1
encoder_states = [state_h_enc, state_c_enc]

In [17]:
encoder_model = Model(encoder_inputs, encoder_states)

In [18]:
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [19]:
dec_emb_layer = model.layers[3]
dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_lstm = model.layers[5]
decoder_outputs2, state_h_dec2, state_c_dec2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h_dec2, state_c_dec2]
decoder_dense = model.layers[6]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Populate the first token of target sequence with the start token.
    target_token = 'starttoken'
    target_seq[0, 0] = tokenizer.word_index.get(target_token, 0)

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tokenizer.index_word.get(sampled_token_index, '')

        if sampled_token == 'endtoken' or len(decoded_sentence) > max_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Test the model with a new input
def correct_grammar(input_sentence):
    input_seq = tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')
    decoded_sentence = decode_sequence(input_seq)
    return decoded_sentence.strip()

In [26]:
import pickle

# Save the model in pickle format
with open('grammar_correction_model.pkl', 'wb') as f:
    pickle.dump(model, f)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>